In [2]:
import os
import dask.dataframe as dd
import featuretools as ft
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
path_profile = r'D:\issp_data\zok_ssp_2017.csv'

log_root = r'D:\issp_data\fe_log_new'
out_root = r'D:\issp_data\features_total'

In [4]:
# loading user profile data
read = lambda p, cs=None: pd.read_csv(p, encoding='shift-jis', usecols=cs)
usecols = [col for col in list(read(path_profile)) if r'名称' not in col]
dropcols = [
    r'モニターID',
    r'暗号化ID(40桁)'
]
columns_jp2en = {
        r'モニターID(11桁)': r'Monitor_ID'
}

df_profile = read(path_profile, usecols)\
                .drop(columns=dropcols)\
                .rename(columns=columns_jp2en)\
                .set_index('Monitor_ID')

In [5]:
file_ls = sorted(os.listdir(log_root))

for month in range(12):
    path_os = os.path.join(log_root, file_ls[month * 4 + 3])
    df_os = dd.read_csv(path_os, compression='zip')\
                .set_index('Monitor_ID')
    print(path_os, df_os.shape)
    
    for cata in range(3):
        file_nm = file_ls[month * 4 + cata]
        path_login = os.path.join(log_root, file_nm)
        file_nm = file_nm[:-9] + r'_features.csv'
        
        print(path_login, r'computing now ...')
        
        dd.read_csv(path_login, compression='zip')\
            .drop(columns='Date')\
            .join(df_os, on='Monitor_ID')\
            .join(df_profile, on='Monitor_ID')\
            .compute()\
            .to_csv(os.path.join(out_root, file_nm), 
                    header=True, index=None, compression='zip')
        
        print(f'>> {file_nm} save file completed!')

D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_os.csv (Delayed('int-0a23e524-e325-4103-aa62-348924ff6c1e'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_0logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_1logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_1_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_2logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_2_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_os.csv (Delayed('int-1ccf6721-e6c0-4907-b67c-95ff34dd38ba'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_0logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_1logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_1_features.csv save file com

In [4]:
path = r'D:\issp_data\features_total\s35809_MobileApp_201701_104628_0_features.csv'
df_os = dd.read_csv(path, compression='zip').compute()

df_os

,Monitor_ID,Day,Month,Weekday,ts_max,ts_mean,ts_median,ts_min,ts_std,ts_var,...,家族人員(0〜2才(乳児)),家族人員(3〜5才(就学前児童)),家族人員(6〜11才(小学生)),家族人員(12〜14才(中学生)),家族人員(15〜17才(高校生)),家族人員(65才以上),自動車(自家用車保有)有無,同居子供人数（17才以下）,同居子供人数（14才以下）,ウエイトセル
0,50000010367,5,1,3,1483658572,1.483615e+09,1.483618e+09,1483577852,26569.735436,7.059508e+08,...,2,2,2,2,2,1,1,1,1,36
1,50000011091,2,1,0,1483348718,1.483349e+09,1.483349e+09,1483348602,40.840135,1.667917e+03,...,2,2,2,2,1,2,1,2,1,125
2,50000011091,16,1,0,1484552063,1.484552e+09,1.484552e+09,1484551003,358.379464,1.284358e+05,...,2,2,2,2,1,2,1,2,1,125
3,50000011730,25,1,2,1485381088,1.485360e+09,1.485370e+09,1485329765,19049.690746,3.628907e+08,...,2,2,2,2,2,1,1,1,1,90
4,50000011763,23,1,0,1485209843,1.485188e+09,1.485185e+09,1485161078,15021.548332,2.256469e+08,...,1,2,2,2,2,2,2,2,2,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316472,50001841730,5,1,3,1483651711,1.483615e+09,1.483616e+09,1483577863,22218.174164,4.936473e+08,...,1,2,2,2,2,2,1,2,2,66
316473,50001841970,29,1,6,1485729375,1.485695e+09,1.485690e+09,1485674463,14757.261257,2.177768e+08,...,2,2,2,2,2,2,2,1,1,64
316474,50001842358,23,1,0,1485209017,1.485176e+09,1.485178e+09,1485150603,18608.005071,3.462579e+08,...,1,2,2,2,2,2,1,2,2,102
316475,50001842551,23,1,0,1485208600,1.485179e+09,1.485186e+09,1485143727,23591.244338,5.565468e+08,...,2,2,2,2,2,2,2,1,1,56
